In [1]:
!sudo apt install pciutils
!sudo apt install lshw
!pip install requests 
!pip install "xinference[transformers]"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 129 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (403 kB/s)    
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline

78Selecting previous

In [6]:
import requests
import os
import zipfile
import tarfile
import subprocess
import os
import threading
import time
import signal
import sys
import stat
import subprocess

class AutoFrp():

    def __init__(self,auto_frp_backend_url:str,platform:str):
        self.auto_frp_backend_url = auto_frp_backend_url
        self.platform = platform
        
    def download_config(self, port: int, output_path: str = "./", remote_port: int = None):
        """
        从AutoFrp服务器下载frpc配置和二进制文件
        
        Args:
            port: 本地要映射的端口
            output_path: 保存下载文件的路径
            remote_port: 可选的远程端口，如果不指定则由服务器随机分配
        
        Returns:
            str: 保存的文件路径
        """

        if not os.path.exists(output_path):
            os.makedirs(output_path)
            
        url = f"{self.auto_frp_backend_url}/config"
        data = {
            "platform": self.platform,
            "port": port
        }
        
        # 如果指定了远程端口，添加到请求数据中
        if remote_port is not None:
            data["remote_port"] = remote_port
        
        response = requests.post(url, json=data)
        
        if response.status_code != 200:
            error_msg = f"下载失败，状态码: {response.status_code}"
            try:
                error_data = response.json()
                if "error" in error_data:
                    error_msg += f", 错误: {error_data['error']}"
            except:
                pass
            raise Exception(error_msg)
        
        # 根据平台确定文件扩展名
        file_ext = "zip" if self.platform == "win" else "tar.gz"
        output_file = os.path.join(output_path, f"frpc.{file_ext}")
        
        with open(output_file, "wb") as f:
            f.write(response.content)
            
        return output_file
    
    def extract_config(self, file_path: str, extract_path: str = "./"):
        """
        解压下载的配置文件
        
        Args:
            file_path: 下载的压缩文件路径
            extract_path: 解压目标路径
            
        Returns:
            str: 解压后的frpc可执行文件路径
        """

        
        if not os.path.exists(extract_path):
            os.makedirs(extract_path)
            
        if self.platform == "win":

            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
            return os.path.join(extract_path, "frpc.exe")
        else:

            with tarfile.open(file_path, 'r:gz') as tar_ref:
                tar_ref.extractall(extract_path)
            frpc_path = os.path.join(extract_path, "frpc")
            # 设置执行权限
            os.chmod(frpc_path, 0o755)
            return frpc_path
    
    def run_frpc(self, frpc_path: str, config_path: str = None, pre_command: str = "xinference-local --host 0.0.0.0 --port 9997 "):
        """
        后台运行frpc客户端并保活
        
        Args:
            frpc_path: frpc可执行文件路径
            config_path: 配置文件路径，默认为同目录下的frpc.toml
            pre_command: 在启动frpc前执行的命令
            
        Returns:
            dict: 包含进程信息和监控进程的字典
        """

        if config_path is None:
            config_path = os.path.join(os.path.dirname(frpc_path), "frpc.toml")
        
        # 在Linux上确保frpc有正确的执行权限
        if self.platform != "win" and os.path.exists(frpc_path):

            current_permissions = os.stat(frpc_path).st_mode
            os.chmod(frpc_path, current_permissions | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)
        
        # 设置进程不显示窗口（仅在Windows上有效）
        startupinfo = None
        if self.platform == "win":

            startupinfo = subprocess.STARTUPINFO()
            startupinfo.dwFlags |= subprocess.STARTF_USESHOWWINDOW
            startupinfo.wShowWindow = 0  # SW_HIDE
        
        # 创建日志文件
        log_path = os.path.join(os.path.dirname(frpc_path), "frpc.log")
        log_file = open(log_path, "w")
        
        # 启动进程
        cmd = [frpc_path, "-c", config_path]
        if pre_command:
            # 构建含有 && 的命令字符串
            cmd_str = f"{pre_command} & {frpc_path} -c {config_path}"
            process = subprocess.Popen(
                cmd_str,
                stdout=log_file,
                stderr=log_file,
                startupinfo=startupinfo,
                creationflags=subprocess.CREATE_NEW_PROCESS_GROUP if self.platform == "win" else 0,
                shell=True
            )
        else:
            process = subprocess.Popen(
                cmd,
                stdout=log_file,
                stderr=log_file,
                startupinfo=startupinfo,
                creationflags=subprocess.CREATE_NEW_PROCESS_GROUP if self.platform == "win" else 0
            )
        
        # 创建保活监控线程
        stop_event = threading.Event()
        
        # 使用可变容器来存储进程引用，避免使用nonlocal
        process_container = {"instance": process}
        
        def keep_alive_monitor():
            while not stop_event.is_set():
                if process_container["instance"].poll() is not None:  # 进程已退出
                    try:
                        # 重新启动进程
                        if pre_command:
                            # 构建含有 && 的命令字符串
                            cmd_str = f"{pre_command} & {frpc_path} -c {config_path}"
                            new_process = subprocess.Popen(
                                cmd_str,
                                stdout=log_file,
                                stderr=log_file,
                                startupinfo=startupinfo,
                                creationflags=subprocess.CREATE_NEW_PROCESS_GROUP if self.platform == "win" else 0,
                                shell=True
                            )
                        else:
                            new_process = subprocess.Popen(
                                cmd,
                                stdout=log_file,
                                stderr=log_file,
                                startupinfo=startupinfo,
                                creationflags=subprocess.CREATE_NEW_PROCESS_GROUP if self.platform == "win" else 0
                            )
                        # 更新进程引用
                        process_container["instance"] = new_process
                        print(f"frpc进程已重启，新PID: {new_process.pid}")
                    except Exception as e:
                        print(f"重启frpc失败: {str(e)}")
                        
                time.sleep(5)  # 每5秒检查一次
        
        # 启动监控线程
        monitor_thread = threading.Thread(target=keep_alive_monitor, daemon=True)
        monitor_thread.start()
        
        # 注册清理函数
        def cleanup_handler():
            stop_event.set()  # 停止监控线程
            if process_container["instance"].poll() is None:  # 如果进程还在运行
                if self.platform == "win":
                    process_container["instance"].send_signal(signal.CTRL_BREAK_EVENT)
                else:
                    process_container["instance"].terminate()
                process_container["instance"].wait(timeout=5)
            log_file.close()

        try:
            # 堵塞主线程，直到捕获到键盘中断（Ctrl+C）
            print(f"frpc进程已启动，PID: {process.pid}，按Ctrl+C停止")
            while not stop_event.is_set():
                time.sleep(1)
        except KeyboardInterrupt:
            print("接收到停止信号，正在停止frpc...")
            cleanup_handler()
            print("frpc已停止")
        
        # 返回包含进程和控制对象的字典（为了保持与原接口兼容）
        return {
            "process": process,
            "stop_monitor": stop_event,
            "log_path": log_path,
            "cleanup": cleanup_handler
        }
    
    def setup_and_run(self, port: int, output_path: str = "./frpc", remote_port: int = None):
        """
        一键设置并运行frpc，此方法会堵塞当前线程直到手动中断（Ctrl+C）
        
        Args:
            port: 本地要映射的端口
            output_path: 保存和解压文件的路径
            remote_port: 可选的远程端口，如果不指定则由服务器随机分配
            pre_command: 在启动frpc前执行的命令，例如 "ollama serve && ollama pull bge-m3"
            
        Returns:
            dict: 包含frpc进程信息和控制函数的字典
        """
        print(f"正在设置映射本地端口 {port} 到 AutoFrp...")
        file_path = self.download_config(port, output_path, remote_port)
        print(f"配置已下载，正在解压...")
        frpc_path = self.extract_config(file_path, output_path)
        print(f"配置已解压，正在启动frpc服务...")
        # 这个调用会堵塞当前线程直到用户中断
        return self.run_frpc(frpc_path)

    


In [ ]:
client=AutoFrp("http://117.158.43.187:28098/","linux")
client.setup_and_run(port=9997,output_path="./frpc_backend",remote_port=7899)



正在设置映射本地端口 9997 到 AutoFrp...
配置已下载，正在解压...
配置已解压，正在启动frpc服务...
frpc进程已启动，PID: 337，按Ctrl+C停止
